In [ ]:
cantidad_especies = 3
cantidad_transcritos = 400
carpeta_data = "./data"

In [ ]:
#Generación de archivos fasta para las 30 especies
import sys
sys.path.append('./libs')
import util_bd, util_fasta
import os
import shutil
from sklearn.externals.joblib import Parallel, delayed, dump

def obtener_especies():
    query = "SELECT * FROM (SELECT CONVERT(@row_number:=@row_number + 1, UNSIGNED) AS orden, m.id_especie, m.especie FROM especies_seleccionadas s JOIN maestra_especies m ON s.especie = m.especie, (SELECT @row_number:=0) AS rn ORDER BY m.id_especie) a ORDER BY 1 LIMIT " + str(cantidad_especies)
    return util_bd.resultados_query(query)
        
def generar_fasta_especie( row_especie):
    os.mkdir(carpeta_data + "/clase_" + str(row_especie[0]))
    # lncRNA
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 0 AND flg_seleccionado = 1 AND id_especie = " + str(row_especie[1]) + " ORDER BY cod_secuencia LIMIT " + str(cantidad_transcritos)
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, carpeta_data + "/clase_" + str(row_especie[0]) + "/lncRNA.fa")
    # PCT
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 1 AND flg_seleccionado = 1 AND id_especie = " + str(row_especie[1]) + " ORDER BY cod_secuencia LIMIT " + str(cantidad_transcritos)
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, carpeta_data + "/clase_" + str(row_especie[0]) + "/PCT.fa")
    # CDS
    query = "SELECT f.cod_secuencia, cds.coding FROM secuencias_CDS cds JOIN secuencias_features f ON cds.id_especie = f.id_especie AND cds.cod_secuencia = f.cod_secuencia WHERE f.flg_pct = 1 AND f.flg_seleccionado = 1 AND f.id_especie = " + str(row_especie[1]) + " ORDER BY f.cod_secuencia LIMIT " + str(cantidad_transcritos)
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, carpeta_data + "/clase_" + str(row_especie[0]) + "/CDS.fa")

if os.path.isdir(carpeta_data):
    shutil.rmtree(carpeta_data)
os.mkdir(carpeta_data)

%time dump(obtener_especies(), carpeta_data + "/info_clases.bin")
%time Parallel(n_jobs=4, verbose=0)(delayed(generar_fasta_especie)(row_especie) for row_especie in obtener_especies())

print("Proceso finalizado")

In [ ]:
# Install a pip package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install keras

In [ ]:
# Install a pip package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install tensorflow

In [ ]:
# Install a conda package in the current Jupyter kernel
#import sys
#!conda install --yes --prefix {sys.prefix} -c anaconda tensorflow-gpu

In [ ]:
# Install a conda package in the current Jupyter kernel
#import sys
#!conda install --yes --prefix {sys.prefix} -c anaconda cudatoolkit

In [ ]:
import sys
sys.path.append('./libs')
import util_caracteristicas
from numpy import array

carpeta_features = "./features"
if os.path.isdir(carpeta_features):
    shutil.rmtree(carpeta_features)
os.mkdir(carpeta_features)
os.mkdir(carpeta_features + "/cpat")
os.mkdir(carpeta_features + "/diamond")

tipo = "train"
archivo_lncRNA = carpeta_data + "/clase_1/lncRNA.fa"
archivo_PCT = carpeta_data + "/clase_1/PCT.fa"
archivo_CDS = carpeta_data + "/clase_1/CDS.fa"
carpeta_cpat = carpeta_features + "/cpat"
util_caracteristicas.generar_modelo_CPAT(archivo_lncRNA, archivo_PCT, archivo_CDS, carpeta_cpat)

archivo_entrada = archivo_lncRNA
diamond_db = "./feature_engine/Diamond_BD/uniprot-viridiplantae-reviewed.dmnd"
archivo_salida = carpeta_features + "/diamond/" + tipo + "_lncRNA.dmnd"
util_caracteristicas.ejecutar_diamond(archivo_entrada, diamond_db, archivo_salida)

archivo_entrada = archivo_PCT
archivo_salida = carpeta_features + "/diamond/" + tipo + "_PCT.dmnd"
util_caracteristicas.ejecutar_diamond(archivo_entrada, diamond_db, archivo_salida)

archivo_entrada = archivo_lncRNA
archivo_salida = carpeta_features + "/cpat/" + tipo + "_lncRNA.cpat"
util_caracteristicas.ejecutar_cpat(archivo_entrada, carpeta_cpat, archivo_salida)

archivo_entrada = archivo_PCT
archivo_salida = carpeta_features + "/cpat/" + tipo + "_PCT.cpat"
util_caracteristicas.ejecutar_cpat(archivo_entrada, carpeta_cpat, archivo_salida)

archivo_entrada = archivo_lncRNA
archivo_cpat = carpeta_features + "/cpat/" + tipo + "_lncRNA.cpat"
archivo_diamond = carpeta_features + "/diamond/" + tipo + "_lncRNA.dmnd"
archivo_salida = carpeta_features + "/" + tipo + "_lncRNA.ft"
util_caracteristicas.generar_features_base(archivo_entrada, archivo_cpat, archivo_diamond, archivo_salida)

archivo_entrada = archivo_PCT
archivo_cpat = carpeta_features + "/cpat/" + tipo + "_PCT.cpat"
archivo_diamond = carpeta_features + "/diamond/" + tipo + "_PCT.dmnd"
archivo_salida = carpeta_features + "/" + tipo + "_PCT.ft"
util_caracteristicas.generar_features_base(archivo_entrada, archivo_cpat, archivo_diamond, archivo_salida)

tipo = "test"
archivo_lncRNA = carpeta_data + "/clase_2/lncRNA.fa"
archivo_PCT = carpeta_data + "/clase_2/PCT.fa"

archivo_entrada = archivo_lncRNA
diamond_db = "./feature_engine/Diamond_BD/uniprot-viridiplantae-reviewed.dmnd"
archivo_salida = carpeta_features + "/diamond/" + tipo + "_lncRNA.dmnd"
util_caracteristicas.ejecutar_diamond(archivo_entrada, diamond_db, archivo_salida)

archivo_entrada = archivo_PCT
archivo_salida = carpeta_features + "/diamond/" + tipo + "_PCT.dmnd"
util_caracteristicas.ejecutar_diamond(archivo_entrada, diamond_db, archivo_salida)

archivo_entrada = archivo_lncRNA
archivo_salida = carpeta_features + "/cpat/" + tipo + "_lncRNA.cpat"
util_caracteristicas.ejecutar_cpat(archivo_entrada, carpeta_cpat, archivo_salida)

archivo_entrada = archivo_PCT
archivo_salida = carpeta_features + "/cpat/" + tipo + "_PCT.cpat"
util_caracteristicas.ejecutar_cpat(archivo_entrada, carpeta_cpat, archivo_salida)

archivo_entrada = archivo_lncRNA
archivo_cpat = carpeta_features + "/cpat/" + tipo + "_lncRNA.cpat"
archivo_diamond = carpeta_features + "/diamond/" + tipo + "_lncRNA.dmnd"
archivo_salida = carpeta_features + "/" + tipo + "_lncRNA.ft"
util_caracteristicas.generar_features_base(archivo_entrada, archivo_cpat, archivo_diamond, archivo_salida)

archivo_entrada = archivo_PCT
archivo_cpat = carpeta_features + "/cpat/" + tipo + "_PCT.cpat"
archivo_diamond = carpeta_features + "/diamond/" + tipo + "_PCT.dmnd"
archivo_salida = carpeta_features + "/" + tipo + "_PCT.ft"
util_caracteristicas.generar_features_base(archivo_entrada, archivo_cpat, archivo_diamond, archivo_salida)

In [ ]:
from sklearn.externals.joblib import load
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.utils import shuffle

features_train_lncRNA = load(carpeta_features + "/train_lncRNA.ft")
features_train_PCT = load(carpeta_features + "/train_PCT.ft")
features_test_lncRNA = load(carpeta_features + "/test_lncRNA.ft")
features_test_PCT = load(carpeta_features + "/test_PCT.ft")

features_train = [list(x.values()) for x in list(features_train_lncRNA.values()) + list(features_train_PCT.values())]
features_test = [list(x.values()) for x in list(features_test_lncRNA.values()) + list(features_test_PCT.values())]
        
x_train = np.array(features_train)
y_train = np.array(([1] * len(features_train_lncRNA)) + ([0] * len(features_train_PCT)))
x_test = np.array(features_test)
y_test = np.array(([1] * len(features_test_lncRNA)) + ([0] * len(features_test_PCT)))

x_train = RobustScaler().fit_transform(x_train)
x_test = RobustScaler().fit_transform(x_test)

x_train, y_train = shuffle(x_train, y_train, random_state=7)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from keras.constraints import maxnorm
from sklearn.model_selection import StratifiedKFold

#config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
#sess = tf.Session(config=config) 
#keras.backend.set_session(sess)

def create_model(optimizer='adam', learn_rate=0.01, momentum=0, init_mode='uniform', activation='relu', activation2='softmax', dropout_rate=0.0, weight_constraint=1, neurons=10):
    #optimizer = SGD(lr=learn_rate, momentum=momentum)
    model = Sequential()
    model.add(Dense(units=10, activation=activation, input_dim=10, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=neurons, activation=activation2, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1, activation='sigmoid', kernel_initializer=init_mode))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
weight_constraint = [1, 2, 3, 4, 5]
neurons = [1, 5, 10, 15, 20, 25, 30]
tuned_parameters = [{'batch_size': [2**5], 'epochs': [2**10], 'optimizer': ['Adam'], 'learn_rate': [0.01], 'momentum': [0], 'init_mode': ['uniform'], 'activation': ['relu'], 'activation': ['softmax'], 'dropout_rate': [0.0], 'weight_constraint': [1], 'neurons': [10]}]

clf = GridSearchCV(model, tuned_parameters, cv=3, scoring=['accuracy','precision','recall'], n_jobs=1, refit="accuracy", return_train_score = True, verbose=0)
%time clf.fit(x_train, y_train)
resultado = {
    "accuracy" : clf.cv_results_['mean_test_accuracy'][clf.best_index_],
    "precision" : clf.cv_results_['mean_test_precision'][clf.best_index_],
    "recall" : clf.cv_results_['mean_test_recall'][clf.best_index_]
}
print(resultado)
print("Best: %f using %s" % (clf.best_score_, clf.best_params_))

#clf = model
#clf.fit(x_train, y_train)

#history = model.fit(x_train, y_train, verbose=0)
#plt.plot(history.history['loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, precision_recall_curve, average_precision_score

y_true = clf.predict(x_test)
print(confusion_matrix(y_true, y_test))
print(classification_report(y_true, y_test, target_names=["lncRNA", "PCT"]))

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

predicted = clf.predict_proba(x_test)
probs = predicted[:,1]
precision, recall, _ = precision_recall_curve(y_test, probs)
average_precision = average_precision_score(y_test, probs)
plt.step(recall, precision, color='b', alpha=0.2, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision))

In [11]:
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())

import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
    
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess)

with tf.Session() as sess:
    print(sess.list_devices())

InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version